# UT6: EDA - Estadística Descriptiva Aplicada

Análisis de datos con Python

# UT6: Análisis Exploratorio de Datos (EDA)

> **Cuaderno de trabajo — UT6: Análisis Exploratorio de Datos (EDA)**
>
> Este notebook contiene los bloques de código y ejercicios de la
> unidad. Para la teoría completa consulta el libro (PDF).
>
> **Requisitos:** ejecuta primero la celda de preparación del entorno y
> la de carga del *dataset*.

## El rol del EDA en el ciclo de vida del proyecto

**EDA inicial vs. EDA profundo:** no son lo mismo.

-   **EDA inicial:** se realiza *antes* de la limpieza, para
    diagnosticar qué problemas tiene el *dataset*.
-   **EDA profundo:** se realiza *después* de la limpieza (es el que
    aborda esta UT), para descubrir relaciones, patrones y supuestos que
    guiarán el modelado.

Confundirlos es uno de los errores más frecuentes en proyectos reales.

## Las 4 preguntas clave del EDA

## Caso de uso: predicción de precios de viviendas

**Dataset House Prices (Ames Housing):** Recopilado por Dean De Cock
para la ciudad de Ames (Iowa, EE. UU.), contiene **1.460 registros** de
ventas de viviendas con **79 variables** que describen prácticamente
cada aspecto de la propiedad: superficie, calidad de materiales, año de
construcción, número de habitaciones, garaje, sótano, etc. La variable
objetivo es `SalePrice` (precio de venta en dólares). Es uno de los
*datasets* de referencia en competiciones de regresión de Kaggle y
volverá a aparecer en UT7, UT9 y UT10.

## Preparación del entorno

In [1]:
# Librerías necesarias para esta unidad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print("Entorno configurado correctamente")

## Preparación del *dataset* House Prices

**Conexión con UT5**

Esta limpieza básica aplica técnicas que viste en la unidad anterior:
imputación de nulos con mediana (variables numéricas) y moda
(categóricas). Aquí lo hacemos de forma compacta para poder centrarnos
en el EDA.

In [2]:
# Dataset Ames Housing: 1.460 viviendas, 79 variables predictoras + Id + SalePrice (81 columnas)
BASE = 'https://raw.githubusercontent.com/jgarcia314/'
url = BASE + 'analisis-datos-python-fp/main/data/raw/house_prices.csv'
df = pd.read_csv(url, na_values=["NA"])
print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")

Dataset cargado: 1460 filas x 81 columnas

**Análisis de los resultados:** El *dataset* contiene 1.460 viviendas y
81 columnas (79 variables predictoras + `Id` + `SalePrice`). Con esta
cantidad de registros y variables, el EDA cobra especial importancia:
sin exploración previa, el riesgo de usar predictores irrelevantes o con
problemas no detectados es muy alto. La limpieza básica que sigue a
continuación es el paso previo necesario antes de cualquier análisis
descriptivo.

In [3]:
# === LIMPIEZA BÁSICA ===
print("\n=== LIMPIEZA BÁSICA ===")

# 1. Inspección inicial de valores nulos
print("\n1. Valores nulos por columna:")
nulos = df.isnull().sum()
print(nulos[nulos > 0])

# 2. Tratamiento de valores nulos
# Para columnas numéricas: imputar con la mediana (vectorizado por columna)
columnas_numericas = df.select_dtypes(include=[np.number]).columns
df[columnas_numericas] = df[columnas_numericas].fillna(df[columnas_numericas].median())

# Para columnas categóricas: imputar con la moda (vectorizado por columna)
columnas_categoricas = df.select_dtypes(include=['object']).columns
df[columnas_categoricas] = df[columnas_categoricas].fillna(
    df[columnas_categoricas].mode().iloc[0]
)

# 3. Verificación
print(f"\nValores nulos después de limpieza: {df.isnull().sum().sum()}")
print(f"\nDataset listo para EDA: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"Variable objetivo: SalePrice")

**Documentación del EDA**

En proyectos profesionales, siempre documenta tu EDA en un Jupyter
Notebook separado. Esto permite que otros (y tú mismo en el futuro)
entiendan las decisiones que tomaste basándote en los datos.

## Análisis univariante: entendiendo variable por variable

### Primera inspección del *dataset*

In [4]:
print("=== INSPECCIÓN DEL DATASET ===")

# El dataset ya está cargado y limpio de la sección anterior
print(f"Forma del *dataset*: {df.shape[0]} filas x {df.shape[1]} columnas")
print("\nPrimeras filas:")
print(df.head())

print("\nInformación general:")
print(df.info())

print("\nTipos de variables:")
print(df.dtypes.value_counts())

### Variables numéricas: más allá de la media

### Medidas de tendencia central revisitadas

In [5]:
print("=== ANÁLISIS DE SALEPRICE (VARIABLE OBJETIVO) ===")

# Las tres medidas de tendencia central
media = df['SalePrice'].mean()
mediana = df['SalePrice'].median()
moda = df['SalePrice'].mode()[0]

print(f"Media: ${media:,.2f}")
print(f"Mediana: ${mediana:,.2f}")
print(f"Moda: ${moda:,.2f}")

print("\n¿Qué nos dicen estas diferencias?")
if media > mediana:
    print("Media > Mediana -> Distribución ASIMÉTRICA A LA DERECHA")
    print("Interpretación: Hay viviendas muy caras que suben la media")
elif media < mediana:
    print("Media < Mediana -> Distribución ASIMÉTRICA A LA IZQUIERDA")
else:
    print("Media = Mediana -> Distribución SIMÉTRICA")

=== ANÁLISIS DE SALEPRICE (VARIABLE OBJETIVO) ===
Media: $180,921.20
Mediana: $163,000.00
Moda: $140,000.00

¿Qué nos dicen estas diferencias?
Media > Mediana -> Distribución ASIMÉTRICA A LA DERECHA
Interpretación: Hay viviendas muy caras que suben la media

**Análisis de los resultados:** La media (\$180.921) supera a la mediana
(\$163.000) en \$17.921 (un 11,0 %), señal clara de asimetría positiva:
existe una cola de viviendas de lujo que eleva el promedio. La moda en
\$140.000 confirma que la mayoría de las transacciones se sitúan por
debajo de la media. En un contexto de negocio inmobiliario, esto implica
que la mediana es el indicador de precio más representativo del mercado
típico, y que cualquier modelo entrenado con la media como referencia
tendería a sobrestimar precios en el segmento mayoritario.

**Recuerda (UT5):** Los *outliers* distorsionan la media de forma
significativa pero apenas afectan a la mediana — por eso la mediana es
la medida de centro recomendada en distribuciones asimétricas o con
valores extremos. Tienes la explicación detallada y el ejemplo numérico
en la **UT5 (Sección 3.2: Detección de outliers)**.

### Medidas de dispersión profundas

| Medida                             | Qué mide                                            | Cuándo usarla                                                 |
|:-----------------|:---------------------|:-------------------------------|
| **Rango**                          | Amplitud total de los datos                         | Visión rápida, muy sensible a *outliers*                      |
| **Varianza**                       | Dispersión promedio al cuadrado respecto a la media | Base para otros cálculos, difícil de interpretar directamente |
| **Desviación estándar**            | Dispersión promedio en unidades originales          | La más usada: mismas unidades que los datos                   |
| **IQR**                            | Amplitud del 50% central de los datos               | Robusta ante *outliers*; ya la usaste en UT5 para detectarlos |
| **Coeficiente de Variación (CV)**  | Dispersión relativa como porcentaje de la media     | Comparar variabilidad entre variables con escalas distintas   |

Medidas de dispersión. Concepto nuevo en esta UT.

In [8]:
print("=== MEDIDAS DE DISPERSIÓN ===")

# Análisis completo de dispersión
precio = df['SalePrice']

print("1. RANGO:")
rango = precio.max() - precio.min()
print(f"   Mínimo: ${precio.min():,.0f}")
print(f"   Máximo: ${precio.max():,.0f}")
print(f"   Rango: ${rango:,.0f}")

print("\n2. VARIANZA Y DESVIACIÓN ESTÁNDAR:")
varianza = precio.var()
std = precio.std()
print(f"   Varianza: {varianza:,.0f}")
print(f"   Desviación estándar: ${std:,.0f}")
print(f"   Interpretación: En promedio, los precios se desvían ${std:,.0f} de la media")

print("\n3. COEFICIENTE DE VARIACIÓN (CV):")
cv = (std / precio.mean()) * 100
print(f"   CV: {cv:.2f}%")
if cv < 15:
    print("   -> Baja variabilidad (datos homogéneos)")
elif cv < 30:
    print("   -> Variabilidad moderada")
else:
    print("   -> Alta variabilidad (datos heterogéneos)")

print("\n4. RANGO INTERCUARTÍLICO (IQR):")
# Como vimos en la UT5, el IQR es una medida de dispersión robusta que no se deja
# distorsionar por valores extremos.
q1 = precio.quantile(0.25)
q3 = precio.quantile(0.75)
iqr = q3 - q1
print(f"   Q1 (25%): ${q1:,.0f}")
print(f"   Q3 (75%): ${q3:,.0f}")
print(f"   IQR: ${iqr:,.0f}")
print(f"   Interpretación: El 50% central de viviendas está en un rango de ${iqr:,.0f}")

=== MEDIDAS DE DISPERSIÓN ===
1. RANGO:
   Mínimo: $34,900
   Máximo: $755,000
   Rango: $720,100

2. VARIANZA Y DESVIACIÓN ESTÁNDAR:
   Varianza: 6,311,111,264
   Desviación estándar: $79,443
   Interpretación: En promedio, los precios se desvían $79,443 de la media

3. COEFICIENTE DE VARIACIÓN (CV):
   CV: 43.91%
   -> Alta variabilidad (datos heterogéneos)

4. RANGO INTERCUARTÍLICO (IQR):
   Q1 (25%): $129,975
   Q3 (75%): $214,000
   IQR: $84,025
   Interpretación: El 50% central de viviendas está en un rango de $84,025

**Análisis de los resultados:** El rango bruto (\$720.100, desde
\$34.900 hasta \$755.000) confirma la presencia de viviendas muy
atípicas que distorsionarán cualquier análisis basado en la media. La
desviación estándar de \$79.443 y el CV del 43,91 % indican alta
heterogeneidad: los precios no son predecibles por la media. El IQR de
\$84.025 (entre \$129.975 y \$214.000) es la medida más fiable: el 50 %
central de las viviendas del *dataset* oscila dentro de esa banda, que
representa el mercado residencial estándar de Ames. Los valores fuera de
ese rango merecen atención especial en el análisis de *outliers*.

El IQR es una medida de dispersión robusta que no se deja distorsionar
por valores extremos. En UT5 lo usamos para *detectar* outliers; aquí lo
usamos para *cuantificar* la variabilidad central del dataset.

In [9]:
print("\n=== EJEMPLO: COMPARANDO VARIABILIDAD ===")

# Calcular CV para diferentes variables
variables_numericas = ['SalePrice', 'GrLivArea', 'YearBuilt', 'OverallQual']

cv_comparacion = (
    df[variables_numericas]
    .agg(lambda col: col.std() / col.mean() * 100)
    .rename("CV (%)")
)
print(cv_comparacion.to_string(float_format="{:.2f}".format))

# Identificar la más variable
var_mas_variable = cv_comparacion.idxmax()
print(f"\nVariable más heterogénea: {var_mas_variable}")


=== EJEMPLO: COMPARANDO VARIABILIDAD ===
SalePrice     43.91
GrLivArea     34.67
YearBuilt      1.53
OverallQual   22.67

Variable más heterogénea: SalePrice

**Análisis de los resultados:** El CV revela diferencias dramáticas
entre variables: `SalePrice` (43,91 %) y `GrLivArea` (34,67 %) son
altamente heterogéneas — el precio y el área habitable varían mucho
entre viviendas —, mientras que `YearBuilt` (1,53 %) es casi constante
en comparación, pues todas las casas se construyeron en rangos
temporales similares. `OverallQual` (22,67 %) muestra variabilidad
moderada. Esta comparación justifica por qué el CV es imprescindible:
sin él, compararíamos desviaciones en dólares, metros cuadrados y años,
magnitudes incomparables entre sí.

**Manos a la Obra**

#### Comparación de medidas de dispersión

**Objetivo:** Entender profundamente cuándo y por qué usar cada medida
de dispersión analizando variables con diferentes escalas y unidades.

**Contexto profesional:** Como analista de riesgos en una consultora
inmobiliaria, debes reportar qué características de las viviendas
presentan mayor volatilidad.

**Instrucciones:**

1.  Analiza estas tres variables: `LotArea` (área del terreno),
    `YearBuilt` (año de construcción) y `GarageArea` (área del garaje).
2.  Para cada una, calcula: Desviación estándar, Rango Intercuartílico
    (IQR) y Coeficiente de Variación (CV).
3.  **Análisis de datos:** Compara el CV de `LotArea` frente al de
    `GarageArea`. ¿Cuál de las dos características es más “impredecible”
    en relación a su magnitud media? Demuestra tu conclusión con los
    números obtenidos.
4.  Explica por qué el CV es la única medida justa para comparar la
    dispersión de `YearBuilt` (escala temporal) con `LotArea` (escala
    espacial).

**Criterio de éxito:** El código debe mostrar una tabla comparativa con
las 3 métricas para las 3 variables. La respuesta a la pregunta 3 debe
estar fundamentada en el cálculo del CV.

**Tiempo estimado:** 15 minutos

In [10]:
# Escribe tu codigo aqui


**Advertencia:** El CV solo tiene sentido para variables con **cero
absoluto** (donde cero = ausencia total). No lo uses cuando el cero es
arbitrario.

-   **Válido:** peso (0 kg = sin peso), altura, ingresos (0€ = sin
    ingresos)
-   **No válido:** temperatura Celsius (0°C es el punto de congelación,
    no “sin temperatura”), años

### Forma de la distribución: asimetría y curtosis

| Tipo                                | Valor Skewness                 | Forma                     | Interpretación                                                       |
|----------|-------------------------|------------|-------------------------|
| **Simétrica**                       | $\approx$ 0 (entre -0.5 y 0.5) | Media $\approx$ Mediana   | Datos equilibrados a ambos lados                                     |
| **Asimétrica positiva** (derecha)   | \> 0.5                         | Cola larga a la derecha   | Mayoría de valores bajos, algunos muy altos (ej: salarios, precios)  |
| **Asimétrica negativa** (izquierda) | \< -0.5                        | Cola larga a la izquierda | Mayoría de valores altos, algunos muy bajos (ej: edad de jubilación) |

Asimetría/Skewness

| Tipo             | Valor Kurtosis             | Forma                    | Interpretación                                    |
|----------|-------------------------|------------|-------------------------|
| **Mesocúrtica**  | $\approx$ 0 (entre -1 y 1) | Similar a la normal      | Comportamiento «típico»                           |
| **Leptocúrtica** | \> 1                       | Pico alto, colas pesadas | Más concentración en el centro + más outliers     |
| **Platicúrtica** | \< -1                      | Más plana, colas ligeras | Datos más uniformemente dispersos, menos outliers |

Curtosis

En Pandas, `.kurt()` devuelve la curtosis relativa a la normal: valor 0
= comportamiento normal, positivo = colas más pesadas, negativo = colas
más ligeras.

In [11]:
print("=== ANÁLISIS DE FORMA DE LA DISTRIBUCIÓN ===")

precio = df['SalePrice']

# Asimetría (Skewness)
skewness = precio.skew()
print(f"ASIMETRÍA (Skewness): {skewness:.2f}")

if abs(skewness) < 0.5:
    print("-> Distribución APROXIMADAMENTE SIMÉTRICA")
elif skewness > 0.5:
    print("-> Distribución ASIMÉTRICA A LA DERECHA (cola derecha larga)")
    print("   Interpretación: Mayoría de valores bajos, algunos muy altos")
else:
    print("-> Distribución ASIMÉTRICA A LA IZQUIERDA (cola izquierda larga)")
    print("   Interpretación: Mayoría de valores altos, algunos muy bajos")

# Curtosis (Kurtosis)
kurtosis = precio.kurt()
print(f"\nCURTOSIS (Kurtosis): {kurtosis:.2f}")

if abs(kurtosis) < 1:
    print("-> MESOCÚRTICA (similar a normal)")
elif kurtosis > 1:
    print("-> LEPTOCÚRTICA (pico alto, colas pesadas)")
    print("   Interpretación: Más datos concentrados en el centro + más *outliers*")
else:
    print("-> PLATICÚRTICA (distribución más plana)")
    print("   Interpretación: Datos más dispersos uniformemente")

=== ANÁLISIS DE FORMA DE LA DISTRIBUCIÓN ===
ASIMETRÍA (Skewness): 1.88
-> Distribución ASIMÉTRICA A LA DERECHA (cola derecha larga)
   Interpretación: Mayoría de valores bajos, algunos muy altos

CURTOSIS (Kurtosis): 6.54
-> LEPTOCÚRTICA (pico alto, colas pesadas)
   Interpretación: Más datos concentrados en el centro + más *outliers*

**Análisis de los resultados:** Con una asimetría (*skewness*) de 1,883,
`SalePrice` presenta asimetría positiva fuerte: la cola de viviendas
caras se extiende hacia la derecha, alejando la media de la mediana. La
curtosis de 6,536 (leptocúrtica) indica colas más pesadas de lo normal,
es decir, hay más *outliers* extremos de los esperados en una
distribución gaussiana. Ambos valores superan los umbrales críticos
(\|skew\| \> 1 y kurtosis \> 3), lo que confirma que una transformación
logarítmica es necesaria antes de aplicar modelos que asumen normalidad
como la regresión lineal. Tras aplicar $\log$, el *skewness* cae a 0,121
y la curtosis a 0,810 — prácticamente normales.

In [14]:
print("\n=== IMPACTO DE LA TRANSFORMACIÓN ===")

# Comparar antes y después de log
precio_original = df['SalePrice']
precio_log = np.log(df['SalePrice'])

print("ANTES de transformación:")
print(f"Skewness: {precio_original.skew():.2f}")

print("\nDESPUÉS de log:")
print(f"Skewness: {precio_log.skew():.2f}")

if abs(precio_log.skew()) < abs(precio_original.skew()):
    print("La transformación logarítmica REDUJO la asimetría")
    print("-> Variable más adecuada para modelos que asumen normalidad")

En proyectos de ML, una regla práctica es aplicar transformación
logarítmica si la asimetría es mayor que 1 o menor que -1.

**Manos a la Obra**

#### Transformaciones y su impacto

**Objetivo:** comprender cuándo y por qué transformar variables
asimétricas.

**Contexto:** estás preparando datos para un modelo de regresión lineal
que asume normalidad. Debes identificar qué variables requieren
transformación y justificar tu decisión.

Analiza la forma de estas variables: `SalePrice`, `GrLivArea` y
`LotArea`.

1.  Para cada una: calcula *skewness* y *kurtosis*, e identifica si es
    simétrica o asimétrica.
2.  Si es asimétrica (`|skew| > 0.5`), aplica transformación logarítmica
    y compara *skewness* antes/después. Verifica visualmente la mejora.
3.  ¿Por qué la transformación logarítmica reduce la asimetría en
    variables con distribución «lognormal»? Explica el mecanismo
    matemático de forma intuitiva.
4.  Comparaste *SalePrice* antes y después de log. Si el *skewness* pasó
    de 1.8 a 0.3, ¿qué significa esto en términos prácticos para un
    modelo de regresión lineal?
5.  ¿Qué pasaría si aplicas log a una variable que ya es simétrica?
    Demuéstralo calculando *skewness* antes/después en *OverallQual*.
6.  Si una variable tiene valores de 0, no puedes aplicar log
    directamente. ¿Qué estrategias alternativas usarías?
7.  En un proyecto real, transformaste *SalePrice* para entrenar el
    modelo. ¿Cómo interpretarías las predicciones del modelo? ¿Qué
    transformación INVERSA aplicarías?

**Criterio de éxito:**

-   Has identificado correctamente qué variables son asimétricas.
-   Has aplicado la transformación solo donde es necesaria.
-   Puedes explicar el impacto de la transformación en números y
    palabras.

Consulta el **Apéndice C.5** para ver cómo calcular la asimetría y
curtosis en Pandas.

``` python
# Escribe aquí tu código
```

**Tiempo estimado:** 15 minutos

No transformes automáticamente todas las variables asimétricas. Pregunta
primero: ¿mi modelo requiere normalidad? Árboles de decisión y modelos
no paramétricos no necesitan transformaciones.

### Implementación completa: función de análisis univariante

In [18]:
def analizar_variable_numerica(df, variable, percentiles=[25, 50, 75]):
    """
    Análisis univariante completo de una variable numérica.

    Parámetros:
    df: DataFrame de Pandas
    variable: nombre de la columna (str)
    percentiles: lista de percentiles a mostrar (default Q1, Q2, Q3)

    Retorna:
    dict con todas las métricas calculadas
    """
    serie = df[variable]

    print(f"\n{'='*60}")
    print(f"ANÁLISIS DE: {variable}")
    print(f"{'='*60}")

    # TENDENCIA CENTRAL
    print("\n1. TENDENCIA CENTRAL:")
    media = serie.mean()
    mediana = serie.median()
    moda = serie.mode()[0] if len(serie.mode()) > 0 else None

    print(f"   Media: {media:.2f}")
    print(f"   Mediana: {mediana:.2f}")
    print(f"   Moda: {moda:.2f}" if moda else "   Moda: No definida")

    # DISPERSIÓN
    print("\n2. DISPERSIÓN:")
    std = serie.std()
    cv = (std / media) * 100 if media != 0 else None
    iqr = serie.quantile(0.75) - serie.quantile(0.25)

    print(f"   Desv. Estándar: {std:.2f}")
    print(f"   Coef. Variación: {cv:.2f}%" if cv else "   CV: No calculable")
    print(f"   IQR: {iqr:.2f}")

    # FORMA
    print("\n3. FORMA:")
    skew = serie.skew()
    kurt = serie.kurt()

    print(f"   Skewness: {skew:.2f}", end=" ")
    if abs(skew) < 0.5:
        print("(Simétrica)")
    elif skew > 0:
        print("(Asimétrica derecha)")
    else:
        print("(Asimétrica izquierda)")

    print(f"   Kurtosis: {kurt:.2f}", end=" ")
    if abs(kurt) < 1:
        print("(Normal)")
    elif kurt > 1:
        print("(Leptocúrtica - colas pesadas)")
    else:
        print("(Platicúrtica - plana)")

    # VALORES EXTREMOS
    print("\n4. VALORES EXTREMOS:")
    print(f"   Min: {serie.min():.2f}")
    print(f"   Max: {serie.max():.2f}")
    print(f"   Rango: {serie.max() - serie.min():.2f}")

    # PERCENTILES
    print(f"\n5. PERCENTILES:")
    for p in percentiles:
        print(f"   P{p}: {serie.quantile(p/100):.2f}")

    # RECOMENDACIÓN
    print("\n6. RECOMENDACIÓN:")
    if abs(skew) > 1:
        print("   - Variable muy asimétrica -> Considerar transformación log")
    if cv and cv > 50:
        print("   - Alta variabilidad -> Analizar *outliers*")
    if kurt > 3:
        print("   - Colas pesadas -> Revisar valores extremos")

    # Retornar diccionario con métricas
    return {
        'media': media,
        'mediana': mediana,
        'std': std,
        'cv': cv,
        'skew': skew,
        'kurt': kurt,
        'min': serie.min(),
        'max': serie.max()
    }

# EJEMPLO DE USO:
metricas_precio = analizar_variable_numerica(df, 'SalePrice')
metricas_area = analizar_variable_numerica(df, 'GrLivArea')

Crea una biblioteca personal de funciones como esta. ¡Te ahorrarán horas
en futuros proyectos!

### Variables categóricas: frecuencias y proporciones

### Tablas de Frecuencia Enriquecidas

In [19]:
print("=== ANÁLISIS DE VARIABLE CATEGÓRICA: NEIGHBORHOOD ===")

# Tabla de frecuencias completa
frecuencias = df['Neighborhood'].value_counts()
proporciones = df['Neighborhood'].value_counts(normalize=True)

# Combinar en un DataFrame informativo
tabla = pd.DataFrame({
    'Frecuencia': frecuencias,
    'Proporción': proporciones,
    'Porcentaje': proporciones * 100
})

tabla['Acumulado %'] = tabla['Porcentaje'].cumsum()

print(tabla.head(10))

print(f"\nTotal de categorías: {len(frecuencias)}")
print(f"Categoría más frecuente: {frecuencias.index[0]} ({frecuencias.iloc[0]} casos)")
print(f"Categoría menos frecuente: {frecuencias.index[-1]} ({frecuencias.iloc[-1]} casos)")

### Concentración de Categorías

In [20]:
print("\n=== ANÁLISIS DE CONCENTRACIÓN ===")

# ¿Cuántas categorías concentran el 80% de los datos?
acumulado = frecuencias / frecuencias.sum()
acumulado_cum = acumulado.cumsum()

n_categorias_80pct = (acumulado_cum <= 0.80).sum()

print(f"Las top {n_categorias_80pct} categorías concentran el 80% de los datos")
print(f"De un total de {len(frecuencias)} categorías")

pct_concentracion = (n_categorias_80pct / len(frecuencias)) * 100
print(f"Nivel de concentración: {pct_concentracion:.1f}%")

if pct_concentracion < 20:
    print("-> Altamente concentrado en pocas categorías")
elif pct_concentracion < 50:
    print("-> Moderadamente concentrado")
else:
    print("-> Distribuido uniformemente")

**Análisis de los resultados:** Si el porcentaje es bajo (menos del
20%), pocas categorías acaparan casi todos los datos. Es la señal para
agrupar las minoritarias bajo una etiqueta “Otros” antes de modelar:
menos ruido, modelos más estables.

**Manos a la Obra**

#### Análisis de concentración categórica

**Objetivo:** Identificar categorías significativas y decidir
estrategias de agrupación basadas en la frecuencia y la
representatividad.

**Contexto profesional:** Debes optimizar el pre-procesamiento de
variables categóricas para evitar el “ruido” estadístico que generan las
clases con muy pocos ejemplos.

**Instrucciones (Semi-guiado):**

1.  Analiza las variables `BldgType` (tipo de edificio) y `Condition1`
    (proximidad a vías principales).
2.  Para ambas, calcula la frecuencia absoluta y el porcentaje que
    representa cada categoría sobre el total.
3.  Identifica cuántas categorías concentran el **80% de los datos**
    (análisis de Pareto).
4.  Localiza aquellas categorías que aparezcan en **menos del 1%** del
    dataset.
5.  **Análisis por datos:** Calcula el precio medio (`SalePrice`) para
    las categorías mayoritarias frente a las minoritarias de `BldgType`.
    ¿Existe una diferencia significativa o las clases raras solo añaden
    complejidad?

**Pistas:**

-   Usa `.value_counts(normalize=True)` para obtener proporciones.
-   Usa `.groupby().agg()` para cruzar con el precio de venta.

**Criterio de éxito:** El código debe identificar las categorías que
concentran el 80% de la muestra y listar aquellas que deberían ser
agrupadas como ‘Otros’ por su baja frecuencia.

**Tiempo estimado:** 20 minutos

In [21]:
# Escribe tu codigo aqui


No agrupes categorías mecánicamente por frecuencia. Primero verifica si
tienen comportamientos similares respecto al target (precio). Agrupa
solo categorías que sean conceptualmente similares Y tengan valores de
target comparables.

### Implementación completa: función de análisis categórico

In [22]:
def analizar_variable_categorica(df, variable, top_n=10):
    """
    Análisis univariante completo de una variable categórica.

    Parámetros:
    df: DataFrame
    variable: nombre de la columna categórica
    top_n: número de categorías principales a mostrar detalladas

    Retorna:
    DataFrame con frecuencias y métricas
    """
    print(f"\n{'='*60}")
    print(f"ANÁLISIS DE: {variable}")
    print(f"{'='*60}")

    # Frecuencias
    frecuencias = df[variable].value_counts()
    n_categorias = len(frecuencias)

    print(f"\n1. INFORMACIÓN GENERAL:")
    print(f"   Total de categorías: {n_categorias}")
    print(f"   Total de valores: {df[variable].notna().sum()}")
    print(f"   Valores nulos: {df[variable].isna().sum()}")

    # Tabla de frecuencias
    tabla = pd.DataFrame({
        'Frecuencia': frecuencias,
        'Porcentaje': (frecuencias / len(df)) * 100
    })
    tabla['Acumulado %'] = tabla['Porcentaje'].cumsum()

    print(f"\n2. TOP {top_n} CATEGORÍAS:")
    print(tabla.head(top_n).to_string())

    # Concentración
    top_80 = (tabla['Acumulado %'] <= 80).sum()
    print(f"\n3. CONCENTRACIÓN:")
    print(f"   Top {top_80} categorías = 80% de datos")
    print(f"   Índice de concentración: {(top_80/n_categorias)*100:.1f}%")

    # Categorías raras
    raras = tabla[tabla['Porcentaje'] < 1.0]
    if len(raras) > 0:
        print(f"\n4. CATEGORÍAS RARAS (< 1%):")
        print(f"   Número de categorías raras: {len(raras)}")
        print(f"   Representan: {raras['Porcentaje'].sum():.2f}% del total")

    # Recomendaciones
    print(f"\n5. RECOMENDACIONES:")
    if n_categorias > 20:
        print(f"   - Muchas categorías ({n_categorias}) -> Considerar agrupación")
    if len(raras) > n_categorias * 0.3:
        print(f"   - {len(raras)} categorías raras -> Agrupar como 'Otros'")
    if top_80 < 5 and n_categorias > 10:
        print(f"   - Alta concentración -> Simplificar variable")

    return tabla

# EJEMPLO DE USO:
tabla_neighborhood = analizar_variable_categorica(df, 'Neighborhood')
tabla_housestyle = analizar_variable_categorica(df, 'HouseStyle')

**Análisis de los resultados:** El reporte impreso resume en segundos lo
que sin la función llevaría varias líneas de código. Fíjate en las
recomendaciones del final: «Considerar agrupación» o «Distribución
equilibrada» son tus próximas tareas concretas en la fase de *feature
engineering*.

Combina esta función con visualizaciones (barplots) para comunicar
efectivamente la distribución de categorías a stakeholders no técnicos.

### Consolidación: análisis univariante

## Análisis bivariante: relaciones entre variables

### Numérica vs. numérica: correlaciones y patrones

### Correlación de Pearson en Profundidad

**Nomenclatura:** Este concepto se conoce por varios nombres
equivalentes:

-   **Coeficiente de correlación** (forma abreviada común)
-   **Coeficiente de correlación de Pearson**
-   **Coeficiente de correlación lineal de Pearson**
-   **r de Pearson** (en notación estadística se representa como *r*)
    $$r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 \sum_{i=1}^{n} (y_i - \bar{y})^2}}$$

Todos se refieren a lo mismo: una medida entre -1 y +1 que cuantifica la
relación lineal entre dos variables. Fue desarrollado por Karl Pearson a
finales del siglo XIX.

In [23]:
print("=== CORRELACIÓN: GRLIVAREA VS SALEPRICE ===")

# Calcular correlación
correlacion = df['GrLivArea'].corr(df['SalePrice'])

print(f"Correlación de Pearson: {correlacion:.3f}")

# Interpretación
print("\nINTERPRETACIÓN:")
if abs(correlacion) < 0.3:
    fuerza = "DÉBIL"
elif abs(correlacion) < 0.7:
    fuerza = "MODERADA"
else:
    fuerza = "FUERTE"

direccion = "POSITIVA" if correlacion > 0 else "NEGATIVA"

print(f"Relación {fuerza} y {direccion}")

if correlacion > 0:
    print("-> A mayor área habitable, mayor precio de venta")
else:
    print("-> A mayor X, menor Y")

# Significancia práctica
print(f"\nSIGNIFICANCIA PRÁCTICA:")
r_cuadrado = correlacion ** 2
print(f"R² = {r_cuadrado:.3f}")
print(f"-> GrLivArea explica el {r_cuadrado*100:.1f}% de la varianza del precio")

**ADVERTENCIA CRÍTICA — La correlación NO es causalidad**

Un ejemplo clásico: las ventas de helados y los casos de ahogamiento
tienen una correlación de r = 0.85. ¿Significa que el helado causa
ahogamientos? **¡NO!**

La explicación es que ambas variables aumentan en verano. El calor es
una **variable de confusión** (*confounding variable*) que afecta a
ambas: más calor → más helados + más gente en piscinas.

**SIEMPRE pregúntate antes de asumir causalidad:**

1.  ¿Hay una explicación causal plausible?
2.  ¿Puede haber una tercera variable que explique ambas?
3.  ¿La correlación se mantiene en diferentes subgrupos?

### Limitaciones de la Correlación de Pearson

**Concepto Crítico:** La correlación de Pearson SOLO detecta relaciones
**LINEALES**. Si la relación es curva o no lineal, Pearson puede dar
resultados engañosos.

In [26]:
print("=== LIMITACIÓN: PEARSON Y RELACIONES NO LINEALES ===")

# Generar relación cuadrática (no lineal)
x = np.linspace(-10, 10, 100)
y = x**2 + np.random.normal(0, 10, 100)  # Parábola con ruido

# Calcular correlación
correlacion_lineal = np.corrcoef(x, y)[0, 1]
print(f"Correlación de Pearson: {correlacion_lineal:.3f}")
print("-> Correlación cercana a 0 aunque hay relación clara (cuadrática)")

print("\nLECCIÓN:")
print("Pearson = 0 NO significa 'no hay relación'")
print("Significa solo 'no hay relación LINEAL'")
print("\nSOLUCIÓN: SIEMPRE visualiza con scatterplot antes de confiar en r")

### Correlación de Spearman: la alternativa robusta

In [27]:
from scipy.stats import spearmanr

print("=== COMPARACIÓN: PEARSON VS SPEARMAN ===")

# Calcular ambas
pearson_r = df['GrLivArea'].corr(df['SalePrice'])
spearman_r, _ = spearmanr(df['GrLivArea'], df['SalePrice'])

print(f"Pearson:  {pearson_r:.3f}")
print(f"Spearman: {spearman_r:.3f}")
print(f"Diferencia: {abs(pearson_r - spearman_r):.3f}")

if abs(pearson_r - spearman_r) > 0.1:
    print("\nDIFERENCIA SIGNIFICATIVA:")
    print("-> Posible relación no lineal o presencia de *outliers*")
    print("-> INVESTIGAR con scatterplot")
else:
    print("\nAMBAS SIMILARES:")
    print("-> Relación aproximadamente lineal")
    print("-> Pearson es suficiente")

=== COMPARACIÓN: PEARSON VS SPEARMAN ===
Pearson:  0.709
Spearman: 0.731
Diferencia: 0.023

AMBAS SIMILARES:
-> Relación aproximadamente lineal
-> Pearson es suficiente

**Análisis de los resultados:** Pearson da 0,709 y Spearman 0,731 para
el par `GrLivArea`-`SalePrice`, con una diferencia de solo 0,023. Al
estar ambas por debajo del umbral de 0,1, la relación es aproximadamente
lineal y Pearson es suficiente en este caso. Si la diferencia hubiera
superado 0,1, habría indicado *outliers* o curvatura que exigirían
investigar con un diagrama de dispersión antes de modelar.

**Cuándo usar cada una:**

**Correlaciones y valores nulos:** `.corr()` omite los nulos en silencio
(*pairwise deletion*): calcula cada par de variables usando solo las
filas donde ambas tienen valor. Si el dataset tiene muchos nulos, dos
correlaciones calculadas sobre muestras muy distintas son difícilmente
comparables. En esta UT los datos ya vienen limpios de UT5, así que no
hay problema — pero tenlo en cuenta cuando trabajes con datos crudos.

### Matrices de correlación

In [28]:
print("=== MATRIZ DE CORRELACIÓN ===")

# Seleccionar variables numéricas relevantes
variables_interes = [
    'SalePrice', 'GrLivArea', 'OverallQual',
    'YearBuilt', 'TotalBsmtSF', 'GarageCars'
]

# Calcular matriz de correlación (numeric_only=True: buena práctica explícita)
matriz_corr = df[variables_interes].corr(numeric_only=True)

print(matriz_corr.round(2))

# Identificar correlaciones fuertes con SalePrice
print("\n=== CORRELACIONES CON SALEPRICE ===")
corr_con_precio = matriz_corr['SalePrice'].sort_values(ascending=False)
print(corr_con_precio)

print("\n=== TOP 3 PREDICTORES ===")
top3 = corr_con_precio[1:4]  # Excluir SalePrice consigo mismo
top3_df = (
    top3
    .rename("r")
    .to_frame()
    .assign(varianza_explicada=lambda df: (df["r"] ** 2 * 100).round(1))
)
print(top3_df.to_string(float_format="{:.3f}".format))

# Detectar multicolinealidad
print("\n=== MULTICOLINEALIDAD (Correlaciones entre predictores) ===")
# Buscar pares con |r| > 0.8 (excluyendo diagonal y triángulo inferior)
umbral = 0.8
corr_vals = (
    matriz_corr
    .where(np.triu(np.ones(matriz_corr.shape, dtype=bool), k=1))
    .stack()
)
pares_multicolineales = corr_vals[corr_vals.abs() > umbral]
for (var1, var2), r in pares_multicolineales.items():
    print(f"{var1} <-> {var2}: r = {r:.3f}")
    print(f"   -> Variables redundantes, considerar eliminar una")

=== MATRIZ DE CORRELACIÓN ===
             SalePrice  GrLivArea  OverallQual  YearBuilt  TotalBsmtSF  \
SalePrice         1.00       0.71         0.79       0.52         0.61   
GrLivArea         0.71       1.00         0.59       0.20         0.45   
OverallQual       0.79       0.59         1.00       0.57         0.54   
YearBuilt         0.52       0.20         0.57       1.00         0.39   
TotalBsmtSF       0.61       0.45         0.54       0.39         1.00   
GarageCars        0.64       0.47         0.60       0.54         0.43   

             GarageCars  
SalePrice          0.64  
GrLivArea          0.47  
OverallQual        0.60  
YearBuilt          0.54  
TotalBsmtSF        0.43  
GarageCars         1.00  

=== CORRELACIONES CON SALEPRICE ===
SalePrice      1.00
OverallQual    0.79
GrLivArea      0.71
GarageCars     0.64
TotalBsmtSF    0.61
YearBuilt      0.52
Name: SalePrice, dtype: float64

=== TOP 3 PREDICTORES ===
                r  varianza_explicada
OverallQual 0.7

**Análisis de los resultados:** El top predictor de `SalePrice` es
`OverallQual` (r = 0,791), seguido de `GrLivArea` (0,709), `GarageArea`
(0,623), `TotalBsmtSF` (0,614) y `YearBuilt` (0,523). Estas cinco
variables explican conjuntamente gran parte de la varianza del precio.
El análisis de multicolinealidad no detecta pares con r \> 0,8 entre
estos predictores, lo que indica que no hay redundancias graves en esta
selección de variables: pueden incluirse todas en un modelo sin riesgo
de coeficientes inestables.

**Manos a la Obra**

#### Análisis de correlaciones y multicolinealidad

**Objetivo:** identificar las variables más predictivas y detectar
redundancias.

**Contexto:** necesitas seleccionar las mejores variables para un modelo
de predicción de precios. Tu jefe te pidió reducir de 20 a 10 variables
sin perder poder predictivo.

Trabaja con estas 12 variables numéricas: `SalePrice`, `GrLivArea`,
`TotalBsmtSF`, `1stFlrSF`, `GarageArea`, `GarageCars`, `TotRmsAbvGrd`,
`OverallQual`, `YearBuilt`, `YearRemodAdd`, `MasVnrArea`, `LotArea`.

1.  Calcula matriz de correlación completa (Pearson).
2.  Identifica las 5 variables MÁS correlacionadas con `SalePrice`.
3.  Detecta pares con multicolinealidad (`|r| > 0.75`). Para cada par
    problemático, decide cuál eliminar.
4.  Compara Pearson vs. Spearman en las *top* 3 variables.
5.  Encontraste que *GarageCars* y *GarageArea* tienen *r = 0.88*. ¿Cuál
    eliminarías y por qué?
6.  Si *YearBuilt* y *YearRemodAdd* tienen *r = 0.52* y *r = 0.51* con
    *SalePrice* respectivamente, pero *r = 0.59* entre ellas, ¿las
    mantendrías ambas?
7.  Supón que *TotalBsmtSF* y *1stFlrSF* tienen *r = 0.82*. ¿Qué
    estrategia alternativa a la eliminación usarías?
8.  Comparaste Pearson vs. Spearman para *OverallQual*. Si Spearman es
    0.80 pero Pearson es 0.75, ¿qué te dice esa diferencia?
9.  En producción real, si una variable tiene *r = 0.85* pero será
    difícil de obtener para casas futuras, ¿la incluirías?

**Criterio de éxito:**

-   Matriz de correlación generada y legible.
-   Has identificado correctamente multicolinealidad.
-   Puedes justificar qué variable conservar de cada par.
-   Has comparado Pearson vs. Spearman en al menos 3 casos.

La matriz de correlación es tu mejor aliada para detectar redundancias.
Tienes los comandos necesarios en el **Apéndice C.5**.

``` python
# Escribe aquí tu código
```

**Tiempo estimado:** 20 minutos

Al detectar multicolinealidad, NO elimines automáticamente la variable
con menor correlación individual con el *target*. Considera también:

-   Facilidad de obtención del dato.
-   Costo de recolección.
-   Estabilidad temporal.
-   Interpretabilidad de negocio.

### Categórica vs. numérica: comparando grupos

### Análisis por grupos con `groupby` avanzado

In [32]:
print("=== PRECIO PROMEDIO POR BARRIO ===")

# Análisis básico
precio_por_barrio = df.groupby('Neighborhood')['SalePrice'].agg([
    ('n', 'count'),
    ('promedio', 'mean'),
    ('mediana', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max')
])

# Ordenar por promedio descendente
precio_por_barrio = precio_por_barrio.sort_values('promedio', ascending=False)

print(precio_por_barrio.head(10))

# Identificar barrios premium
promedio_global = df['SalePrice'].mean()
precio_por_barrio['vs_promedio'] = \
    ((precio_por_barrio['promedio'] - promedio_global) / promedio_global) * 100

print("\n=== BARRIOS PREMIUM (>25% sobre promedio) ===")
premium = precio_por_barrio[precio_por_barrio['vs_promedio'] > 25]
print(premium[['promedio', 'vs_promedio']])

print("\n=== BARRIOS ECONÓMICOS (<-25% bajo promedio) ===")
economicos = precio_por_barrio[precio_por_barrio['vs_promedio'] < -25]
print(economicos[['promedio', 'vs_promedio']])

=== PRECIO PROMEDIO POR BARRIO ===
                n   promedio   mediana        std     min     max
Neighborhood                                                     
NoRidge        41  335295.32  301500.0  121412.66  190000  755000
NridgHt        77  316270.62  315000.0   96392.54  154000  611657
StoneBr        25  310499.00  278000.0  112969.68  170000  556581
Timber         38  242247.45  228475.0   64845.65  137500  378500
Veenker        11  238772.73  218000.0   72369.32  162500  385000
Somerst        86  225379.84  225500.0   56177.56  144152  423000
ClearCr        28  212565.43  200250.0   50231.54  130000  328000
Crawfor        51  210624.73  200624.0   68866.40   90350  392500
CollgCr       150  197965.77  197200.0   51403.67  110000  424870
Blmngtn        17  194870.88  191000.0   30393.23  159895  264561

=== BARRIOS PREMIUM (>25% sobre promedio) ===
               promedio  vs_promedio
Neighborhood                        
NoRidge       335295.32        85.33
NridgHt       3

**Análisis de los resultados:** El precio por barrio varía de forma
dramática: NoRidge (\$335.295), NridgHt (\$316.271) y StoneBr
(\$310.499) lideran el segmento *premium*, mientras que MeadowV
(\$98.576) queda en el extremo opuesto. La ratio máximo/mínimo es de
3,4x, lo que confirma que `Neighborhood` es una de las variables
categóricas con mayor poder predictivo del *dataset*. Un modelo que no
incluya el barrio como variable estará sistemáticamente sesgado:
infraestimará precios en zonas *premium* y los sobreestimará en las
económicas.

No te quedes solo con la media. La desviación estándar te dice si los
precios en ese barrio son homogéneos o muy variables. Alta *std* =
barrio mixto (casas de diferentes calidades).

### Tablas de contingencia simples

In [33]:
print("=== DISTRIBUCIÓN DE CALIDAD POR BARRIO ===")

# Tabla de contingencia
tabla_contingencia = pd.crosstab(
    df['Neighborhood'],
    df['OverallQual'],
    margins=True,
    margins_name='Total'
)

print(tabla_contingencia.head(10))

# Convertir a proporciones
tabla_proporciones = pd.crosstab(
    df['Neighborhood'],
    df['OverallQual'],
    normalize='index'  # Normalizar por fila (dentro de cada barrio)
)

print("\n=== PROPORCIONES (dentro de cada barrio) ===")
print((tabla_proporciones * 100).round(1).head())

# Identificar barrios donde >50% tienen calidad 9-10
barrios_calidad_alta = tabla_proporciones[
    (tabla_proporciones[9] + tabla_proporciones[10]) > 0.5
]
print(f"\nBarrios donde >50% tienen calidad 9-10:")
print(barrios_calidad_alta.index.tolist())

**Manos a la Obra**

#### Análisis de segmentación por categorías

**Objetivo:** descubrir cómo variables categóricas impactan al *target*
y detectar segmentos de mercado.

**Contexto:** eres analista de una inmobiliaria y necesitas identificar
segmentos de mercado para una campaña de marketing dirigida. El CMO
quiere saber: “¿qué características definen el mercado *premium*
vs. económico?”.

Realiza análisis de grupos para: `Neighborhood` (barrio), `HouseStyle`
(estilo de casa) y `OverallQual` (calidad general).

1.  Para `Neighborhood`: calcula *n*, promedio, mediana, *std*, min,
    max. Identifica *top* 5 barrios *premium* y *bottom* 5 económicos.
    Calcula desviación vs. promedio global.
2.  Para `HouseStyle`: identifica estilos con mayor variabilidad de
    precio.
3.  Para `OverallQual`: analiza si la relación es lineal e identifica el
    umbral donde el precio se dispara.
4.  Crea una tabla de contingencia: `Neighborhood` vs. `OverallQual`.
    Identifica barrios donde predominan casas de baja calidad.
5.  Si el barrio “OldTown” tiene un precio promedio de 120 k\$ con
    $s = 40\text{ k\$}$, mientras “StoneBr” tiene 300 k\$ con
    $s = 80\text{ k\$}$, ¿cuál es MÁS homogéneo? Calcula el CV.
6.  Encontraste que los estilos “1Story” y “2Story” tienen precios
    promedio similares pero diferentes medianas. ¿Qué te dice esa
    discrepancia?
7.  Si la calidad 5 tiene un promedio de 140 k\$ y la calidad 8 tiene
    230 k\$, ¿es la relación lineal? ¿Qué implicaciones tiene para
    *feature engineering*?
8.  En tu tabla de contingencia encontraste que el barrio “MeadowV”
    tiene un 80 % de casas con calidad $\leq 5$. ¿Lo mantendrías como
    categoría separada o lo agruparías?
9.  Imagina que tu modelo predice el precio basándose en el barrio. En
    producción, aparece una casa en un barrio nuevo. ¿Qué estrategia
    implementarías?

**Criterio de éxito:**

-   Has usado `.groupby().agg()` con múltiples funciones.
-   Has calculado la desviación porcentual vs. promedio global.
-   Has identificado segmentos de mercado claramente diferenciados.
-   Has generado al menos una tabla de contingencia interpretable.

El análisis por grupos con `.groupby()` es fundamental para segmentar el
mercado. Tienes las funciones de agregación en el **Apéndice C.5**.

``` python
# Escribe aquí tu código
```

**Tiempo estimado:** 20 minutos

In [35]:
# ESTRUCTURA SUGERIDA (sin código completo):
resultados = df.groupby('Variable_Categorica')['SalePrice'].agg({
    'n': 'count',
    'promedio': 'mean',
    # ... añade más según necesites
})

Al presentar los resultados a *stakeholders*, ordena SIEMPRE por
promedio descendente o ascendente. Una tabla ordenada alfabéticamente no
comunica el *insight* de “qué barrios son *premium*”.

### Categórica vs. categórica: tablas de contingencia

In [36]:
print("=== RELACIÓN: GARAGETYPE VS HOUSESTYLE ===")

# Crear tabla de contingencia
tabla = pd.crosstab(
    df['GarageType'],
    df['HouseStyle'],
    margins=True
)

print(tabla)

# Convertir a proporciones por fila
tabla_prop = pd.crosstab(
    df['GarageType'],
    df['HouseStyle'],
    normalize='index'
)

print("\n=== PROPORCIONES (por tipo de garaje) ===")
print((tabla_prop * 100).round(1))

# Análisis
print("\nINTERPRETACIÓN:")
print("Si hay asociación, las proporciones variarán entre filas")
print("Si son independientes, las proporciones serán similares")

**Análisis de los resultados:** Mira las proporciones por fila. Si
cambian mucho de una fila a otra, las dos variables están asociadas: el
tipo de garaje influye en el estilo de vivienda. Si las proporciones son
parecidas en todas las filas, son independientes y puedes tratarlas por
separado.

### Concepto Intuitivo de Independencia

In [37]:
print("=== DETECTAR INDEPENDENCIA VISUALMENTE ===")

# Si las proporciones son MUY similares entre grupos -> Independencia
# Si varían mucho -> Asociación

tabla_prop = pd.crosstab(
    df['CentralAir'],  # Aire acondicionado (Y/N)
    df['Heating'],     # Tipo de calefacción
    normalize='index'
)

print(tabla_prop * 100)

# Calcular variabilidad de proporciones (vectorizado por columna)
variabilidad = tabla_prop.std().rename("std_proporciones")
etiquetas = variabilidad.apply(
    lambda v: "posible independencia" if v < 0.05 else "hay asociación"
).rename("interpretación")
print(pd.concat([variabilidad.round(4), etiquetas], axis=1).to_string())

**Manos a la Obra**

#### Análisis de independencia entre categóricas

**Objetivo:** Determinar mediante análisis de contingencia si existe una
asociación estadística relevante entre diferentes características
cualitativas de las viviendas.

**Contexto profesional:** Como parte de un proceso de selección de
características (*feature selection*), debes identificar variables
categóricas redundantes (altamente asociadas) para simplificar el modelo
final y evitar multicolinealidad cualitativa.

**Tu misión (autónomo):**

1.  Selecciona dos pares de variables categóricas que creas que podrían
    estar relacionadas (ej: `GarageType` vs `GarageFinish`) y otro par
    que creas que debería ser independiente.
2.  Genera las tablas de contingencia normalizadas por filas para ambos
    casos.
3.  **Juicio profesional:** Analiza los resultados y determina,
    basándote exclusivamente en los datos obtenidos, qué par presenta
    una asociación real y cuál parece ser independiente. Demuestra tu
    conclusión señalando cómo varían las proporciones en las celdas de
    la tabla.
4.  Justifica cuál de las variables eliminarías del modelo si
    encontraras una asociación superior al 90%.

**Criterio de éxito:** Presentar dos tablas de contingencia legibles y
una conclusión escrita que identifique correctamente la presencia o
ausencia de asociación basada en la variabilidad de las frecuencias
relativas.

**Tiempo estimado:** 15 minutos

In [38]:
# Escribe tu codigo aqui


Si trabajas con más de 2 variables categóricas, considera usar análisis
de correspondencias múltiples (MCA) o chi-cuadrado de forma sistemática.
Para 2-3 pares, el análisis visual de tablas es suficiente y más
interpretable.

### Consolidación: análisis bivariante

## Análisis multivariante: explorando interacciones

### Estratificación: Análisis Condicional

In [39]:
print("=== CORRELACIÓN GrLivArea-SalePrice ESTRATIFICADA POR BARRIO ===")

# Seleccionar top 5 barrios por frecuencia
# Barrios con suficientes muestras para una correlación fiable (n >= 20)
barrios_validos = df['Neighborhood'].value_counts()
barrios_validos = barrios_validos[barrios_validos >= 20].index

# Calcular correlación estratificada por barrio (vectorizado con groupby+apply)
def corr_grlivarea_saleprice(sub):
    return sub['GrLivArea'].corr(sub['SalePrice'])

df_resultados = (
    df[df['Neighborhood'].isin(barrios_validos)]
    .groupby('Neighborhood')
    .apply(corr_grlivarea_saleprice, include_groups=False)
    .rename("Correlación")
    .to_frame()
    .assign(n=df[df['Neighborhood'].isin(barrios_validos)].groupby('Neighborhood').size())
    .sort_values('Correlación', ascending=False)
)
print(df_resultados.to_string(float_format="{:.3f}".format))

print("\n=== ANÁLISIS ===")
print(f"Correlación mínima: {df_resultados['Correlación'].min():.3f}")
print(f"Correlación máxima: {df_resultados['Correlación'].max():.3f}")
print(f"Rango: {df_resultados['Correlación'].max() - df_resultados['Correlación'].min():.3f}")

if df_resultados['Correlación'].std() > 0.15:
    print("\nHAY INTERACCIÓN:")
    print("La relación GrLivArea-SalePrice VARÍA según el barrio")
    print("-> Considera crear features de interacción: GrLivArea * Neighborhood")
else:
    print("\nNO HAY INTERACCIÓN:")
    print("La relación es consistente entre barrios")

### Agregaciones múltiples y *pivoting*

In [40]:
print("=== PRECIO PROMEDIO POR BARRIO Y CALIDAD ===")

# Pivot table: Barrio (filas) x Calidad (columnas)
pivot = df.pivot_table(
    values='SalePrice',
    index='Neighborhood',
    columns='OverallQual',
    aggfunc='mean'
)

# Mostrar solo top barrios y calidades relevantes
barrios_relevantes = df['Neighborhood'].value_counts().head(8).index
calidades_relevantes = [5, 6, 7, 8, 9, 10]

pivot_filtrado = pivot.loc[barrios_relevantes, calidades_relevantes]

print(pivot_filtrado.round(0))

# Análisis de patrones (vectorizado por barrio con apply)
print("\n=== PATRONES DETECTADOS ===")

def analizar_patron(serie):
    incrementos = serie.dropna().sort_index().diff().dropna()
    if len(incrementos) < 2:
        return pd.Series({'incremento_promedio': float('nan'), 'std_incremento': float('nan'), 'no_lineal': False})
    media = incrementos.mean()
    std = incrementos.std()
    return pd.Series({'incremento_promedio': media, 'std_incremento': std, 'no_lineal': (std / media > 0.5) if media else False})

patrones = (
    pivot.loc[barrios_relevantes]
    .apply(analizar_patron, axis=1)
)
print(patrones.to_string(float_format='{:,.0f}'.format))

**Análisis de los resultados:** La columna `incremento_promedio` indica
cuántos euros añade de media cada punto extra de calidad en ese barrio.
En NridgHt cada punto vale ~57.700 €, mientras que en Sawyer solo ~8.700
€: la calidad “pesa” 6,6 veces más en el barrio *premium*. La columna
`std_incremento` mide si ese incremento es regular o errático: en
Edwards la desviación (91.503 €) supera al incremento medio (15.911 €),
lo que indica saltos muy irregulares — posiblemente por pocas
observaciones en alguna categoría o por NaN en la tabla pivotada. La
columna `no_lineal` resume ambas señales: todos los barrios con muestra
suficiente muestran no linealidad. Eso confirma que un modelo lineal que
trate calidad y barrio como efectos independientes estará mal
especificado; necesitarás una variable de interacción
`OverallQual × Neighborhood` o un modelo basado en árboles que la
capture automáticamente.

### *Pivot tables* avanzadas: múltiples agregaciones

In [41]:
print("=== PIVOT TABLE CON MÚLTIPLES AGREGACIONES ===")

# Analizar precio por barrio y calidad con varias métricas
pivot_completo = df.pivot_table(
    values='SalePrice',
    index='Neighborhood',
    columns='OverallQual',
    aggfunc=['mean', 'count', 'std']
)

# Acceder a diferentes agregaciones
print("PROMEDIO:")
print(pivot_completo['mean'].head())

print("\nCONTEO (n por celda):")
print(pivot_completo['count'].head())

# Identificar celdas con pocos datos (n < 5)
conteos = pivot_completo['count']
celdas_problematicas = conteos < 5

print(f"\n=== CELDAS CON DATOS INSUFICIENTES (n<5) ===")
print(f"Total de combinaciones: {conteos.size}")
print(f"Combinaciones con n<5: {celdas_problematicas.sum().sum()}")
print(f"Porcentaje problemático: {(celdas_problematicas.sum().sum() / conteos.size) * 100:.1f}%")

**Análisis de los resultados:** Con 250 combinaciones posibles (25
barrios × 10 niveles de calidad) y solo 1.460 viviendas, era inevitable
que muchas celdas quedaran vacías o con muy pocos casos. El output lo
confirma: el 17,6 % de las celdas tienen n \< 5 y muchas directamente
son NaN. Eso no invalida el análisis, pero sí obliga a interpretarlo con
cautela: un promedio basado en 2 o 3 observaciones es muy sensible a
outliers. La tabla de conteos es tan importante como la de promedios —
léelas siempre en paralelo. Si el porcentaje problemático superara el
30-40 %, lo adecuado sería reducir la granularidad: por ejemplo, agrupar
los 10 niveles de calidad en tres categorías (baja 1-4, media 5-7, alta
8-10) antes de pivotar.

**Manos a la Obra**

#### Análisis de interacciones multivariante

**Objetivo:** Detectar interacciones entre variables que no son visibles
mediante análisis bivariante simple y cuantificar su impacto en el
precio.

**Contexto profesional:** Tu director comercial te plantea una
hipótesis: *“Creo que los metros cuadrados no se valoran igual en un
barrio humilde que en uno de lujo”*. Tu tarea es demostrar o desmentir
esta hipótesis con evidencias estadísticas.

**Tu misión (autónomo):**

1.  Selecciona los **3 barrios con más registros** del dataset.
2.  Calcula la correlación entre la superficie habitable (`GrLivArea`) y
    el precio (`SalePrice`) de forma independiente para cada uno de esos
    3 barrios.
3.  Crea una tabla dinámica (`pivot_table`) que muestre el precio medio
    cruzando el barrio (`Neighborhood`) con la calidad general
    (`OverallQual`) para los barrios seleccionados.
4.  **Informe de hallazgos:** ¿Es constante la relación entre metros
    cuadrados y precio en los tres barrios? Identifica una combinación
    de Barrio/Calidad donde el precio medio sea inusualmente alto o bajo
    en comparación con la tendencia general.

**Criterio de éxito:** Presentar las 3 correlaciones diferenciadas y la
tabla dinámica. La conclusión debe responder explícitamente a la
hipótesis del director comercial usando los datos obtenidos.

**Tiempo estimado:** 25 minutos

In [42]:
# Escribe tu codigo aqui


En modelos tree-based (Random Forest, XGBoost), las interacciones se
capturan automáticamente. En regresión lineal, debes crearlas
manualmente. Esto afecta tu estrategia de *feature engineering*.

### Consolidación: análisis multivariante

## Estadística inferencial: para ir más allá

**Para ir más allá — Tests de hipótesis y p-valores**

El EDA que has aprendido en esta UT cubre el análisis exploratorio de
patrones. Cuando necesites **confirmar estadísticamente** si una
diferencia entre grupos es real o podría ser fruto del azar (A/B
testing, validación de modelos, investigación), entra en juego la
*estadística inferencial*.

Las herramientas clave están en `scipy.stats`: `ttest_ind()` para
comparar medias de dos grupos, `mannwhitneyu()` como alternativa no
paramétrica, y `chi2_contingency()` para variables categóricas. Para el
80% de los proyectos en tu primer empleo, el EDA de esta UT es
suficiente.

## Metodología de EDA: flujo de trabajo profesional

### *Checklist* del EDA completo

Usa este checklist para asegurar que no omites pasos críticos:

**FASE 1: PRIMERA INSPECCIÓN (15 min)**

-   [ ] Cargar datos y verificar que se cargaron correctamente
-   [ ] Verificar dimensiones con .shape
-   [ ] Inspeccionar primeras y últimas filas con .head() y .tail()
-   [ ] Revisar tipos de datos con .dtypes
-   [ ] Verificar valores nulos con .isnull().sum()
-   [ ] Obtener resumen estadístico con .describe()

**FASE 2: ANÁLISIS UNIVARIANTE (30-60 min)**

Numéricas:

-   [ ] Calcular tendencia central (media, mediana, moda)
-   [ ] Calcular dispersión (std, CV, IQR)
-   [ ] Calcular forma (skewness, kurtosis)
-   [ ] Identificar necesidad de transformaciones
-   [ ] Crear histogramas para variables clave

Categóricas:

-   [ ] Calcular frecuencias y proporciones
-   [ ] Identificar categorías raras (\< 1%)
-   [ ] Analizar concentración (regla 80/20)
-   [ ] Decidir estrategia de agrupación si necesaria

**FASE 3: ANÁLISIS BIVARIANTE (45-90 min)**

Numérica-Numérica:

-   [ ] Calcular matriz de correlación
-   [ ] Identificar top 5 predictores del target
-   [ ] Detectar multicolinealidad (\|r\| \> 0.75)
-   [ ] Comparar Pearson vs Spearman en casos dudosos
-   [ ] Crear scatterplots para relaciones fuertes

Categórica-Numérica:

-   [ ] Análisis por grupos (.groupby)
-   [ ] Identificar categorías con mayor/menor target
-   [ ] Calcular variabilidad dentro de categorías
-   [ ] Detectar segmentos de mercado

Categórica-Categórica:

-   [ ] Tablas de contingencia para pares relevantes
-   [ ] Evaluar independencia visualmente
-   [ ] Identificar asociaciones fuertes

**FASE 4: ANÁLISIS MULTIVARIANTE (60-120 min)**

-   [ ] Estratificar correlaciones por subgrupos
-   [ ] Crear pivot tables multidimensionales
-   [ ] Detectar interacciones clave (efecto diferencial)
-   [ ] Analizar no-linealidades y umbrales

**FASE 5: DOCUMENTACIÓN (30 min)**

-   [ ] Crear sección de hallazgos clave
-   [ ] Documentar decisiones de transformación
-   [ ] Listar variables a eliminar/agrupar
-   [ ] Anotar hipótesis para feature engineering
-   [ ] Preparar 2-3 visualizaciones clave para stakeholders

### Registro de hallazgos clave

**Buena práctica:** Mantén un “notebook de hallazgos” paralelo donde
documentes los resultados clave. El siguiente bloque genera
automáticamente ese registro a partir de los datos reales:

  HALLAZGOS CLAVE — House Prices Dataset

## 1. Variables más predictivas (|r| con SalePrice)
   OverallQual          r=0.79
   GrLivArea            r=0.71
   GarageCars           r=0.64
   GarageArea           r=0.62
   TotalBsmtSF          r=0.61

## 2. Candidatas a transformación logarítmica (skew > 1)
   SalePrice            skew=+1.88  log_skew=+0.12  -> aplicar log
   LotArea              skew=+12.21  log_skew=-0.14  -> aplicar log
   GrLivArea            skew=+1.37  log_skew=-0.01  -> aplicar log
   TotalBsmtSF          skew=+1.52  log_skew=-5.15  -> aplicar log
   1stFlrSF             skew=+1.38  log_skew=+0.08  -> aplicar log

## 3. Multicolinealidad detectada (|r| > 0.8 entre predictores)
   GarageCars <-> GarageArea  r=0.88  -> considerar eliminar GarageArea
   GrLivArea <-> TotRmsAbvGrd  r=0.83  -> considerar eliminar TotRmsAbvGrd
   TotalBsmtSF <-> 1stFlrSF  r=0.82  -> considerar eliminar 1stFlrSF

## 4. Categorías raras a agrupar (n < 10)
   Neighborhood:
      "NPkVill" 

Esta documentación se genera automáticamente con los datos reales y te
ahorrará horas cuando vuelvas al proyecto semanas después o cuando
presentes resultados a tu equipo.

### *Template* de *notebook* de EDA

Añade celdas de Markdown entre bloques de código para explicar QUÉ estás
buscando y POR QUÉ es relevante. Un notebook sin narrativa es solo
código difícil de seguir.

### Preparación para modelado: *checklist* final

Antes de pasar a entrenar modelos (UT8), verifica:

**Variables:**

-   [ ] Variables finales seleccionadas documentadas
-   [ ] Multicolinealidad resuelta (\|r\| \< 0.8 entre predictores)
-   [ ] Categorías raras agrupadas o eliminadas

**Transformaciones:**

-   [ ] Variables asimétricas transformadas (log, sqrt, etc.)
-   [ ] Transformaciones documentadas (para invertir predicciones)
-   [ ] Escalado no necesario aún (se hace en UT8)

**Datos limpios:**

-   [ ] Nulos manejados (de UT5)
-   [ ] Outliers revisados y decisión tomada
-   [ ] Duplicados eliminados (de UT5)

**Split Train/Test:**

-   [ ] Train/Test definido (80/20)
-   [ ] Estratificado si hay desbalance
-   [ ] Sin data leakage (split ANTES de imputación/scaling)

**Documentación:**

-   [ ] Decisiones justificadas con EDA
-   [ ] Supuestos documentados
-   [ ] Baseline definido

### Errores comunes y cómo evitarlos

## Ejemplos de referencia: patrones EDA

### Análisis Univariante (Numérico y Categórico)

In [46]:
import pandas as pd
import numpy as np

# 1. Medidas de tendencia y dispersión robustas
stats = df['col'].agg(['mean', 'median', 'std'])
cv = df['col'].std() / df['col'].mean() # Coeficiente de Variación

# 2. Concentración categórica (frecuencias)
freq = df['cat'].value_counts()
rel_freq = df['cat'].value_counts(normalize=True) * 100

### Análisis Bivariante y Correlaciones

In [47]:
# 1. Matriz de correlación de Pearson
corr_matrix = df.corr(numeric_only=True)

# 2. Análisis de contingencia (Categórica vs Categórica)
contingencia = pd.crosstab(df['cat1'], df['cat2'])

# 3. Segmentación (Categórica vs Numérica)
segmentos = df.groupby('cat')['num'].describe()

### Transformaciones y Multivariante

In [48]:
# 1. Transformación Logarítmica (reduce asimetría)
df['log_val'] = np.log1p(df['val'])

# 2. Agregaciones multivariante
multi_stats = df.pivot_table(values='num', index='cat1', columns='cat2', aggfunc='mean')

## Conceptos Clave

-   **Asimetría (Skewness):** Medida que indica si los datos están
    distribuidos de forma equilibrada respecto a la media o si se
    concentran en un extremo.
-   **Curtosis:** Medida que indica el grado de concentración de los
    datos alrededor de la zona central de la distribución
    (“puntiagudez”).
-   **Correlación:** Relación estadística entre dos variables que indica
    cómo varía una al cambiar la otra (no implica causalidad).
-   ***p-valor*:** valor que ayuda a determinar la significancia
    estadística de un hallazgo; si es bajo, es poco probable que el
    resultado sea azaroso.

### Checklist de autoevaluación

Antes de pasar a la práctica, asegúrate de que puedes:

-   [ ] Interpretar qué indica una asimetría (skewness) positiva o
    negativa.
-   [ ] Diferenciar entre una correlación fuerte y una relación de
    causalidad.
-   [ ] Interpretar el *p-valor* para validar o rechazar una hipótesis.
-   [ ] Visualizar la curtosis para entender la frecuencia de valores
    extremos.

## Fuentes y Lecturas Recomendadas

**¿Quieres profundizar más?** Consulta la bibliografía detallada, los
enlaces a la documentación oficial y los recursos de aprendizaje para
esta unidad en el **Apéndice B: Fuentes y Lecturas Recomendadas** al
final de este libro.

### Reflexión final